This file contains code for fine-tuning the model using Gradient AI. This fine-tuning code is not integrated with the final deployment file because of gradientai and gradio dependency issues as these both libraries contradict the pydantic dependency. Therefore, this code is present here for the TA's to see and play around with to see that this fine-tuning approach works as well.

In [1]:
!pip install --upgrade pydantic gradientai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 18.0 MB/s eta 0:00:00
  

In [5]:
import pandas as pd
import os
from gradientai import Gradient

def create_model_adapter(gradient):
    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")
    new_model_adapter = base_model.create_model_adapter(
        name="nous-hermes2"
    )
    print(f"Created model adapter with id {new_model_adapter.id}")
    return new_model_adapter

def pre_fine_tune_and_save():
    os.environ['GRADIENT_WORKSPACE_ID'] = '9d0447f2-fcd4-4177-9145-9f019fd59f1e_workspace'
    os.environ['GRADIENT_ACCESS_TOKEN'] = 'cPErsUMgadGMbzeq8z8W36eJn7UA0Uob'

    df = pd.read_csv("https://raw.githubusercontent.com/CS-5302/CS-5302-Project-Group-15/main/Datasets/testing/combined_df.csv")
    BATCH_SIZE = 100
    NUM_EPOCHS = 1
    gradient = Gradient()

    new_model_adapter = create_model_adapter(gradient)

    # Split the DataFrame into batches and fine-tune
    batches = [df[i:i + BATCH_SIZE] for i in range(0, len(df), BATCH_SIZE)]
    for batch_index, batch in enumerate(batches):
        fine_tuning_samples = [{'inputs': f"### Instruction: {row['prompts']}", 'targets': f"### Response: {row['results']}"} for _, row in batch.iterrows()]
        for epoch in range(NUM_EPOCHS):
            print(f"Fine-tuning batch {batch_index + 1} (epoch {epoch + 1})")
            new_model_adapter.fine_tune(samples=fine_tuning_samples)

    # Save the adapter ID or state
    model_id = new_model_adapter.id  # Assuming you can save ID or use a method to save state
    return model_id

# Run this function once to pre-fine-tune and save the model state
model_id = pre_fine_tune_and_save()

Created model adapter with id bea513a0-b418-4442-8ca1-c5861f851ff6_model_adapter
Fine-tuning batch 1 (epoch 1)
Fine-tuning batch 2 (epoch 1)
Fine-tuning batch 3 (epoch 1)
Fine-tuning batch 4 (epoch 1)
Fine-tuning batch 5 (epoch 1)
Fine-tuning batch 6 (epoch 1)
Fine-tuning batch 7 (epoch 1)
Fine-tuning batch 8 (epoch 1)
Fine-tuning batch 9 (epoch 1)
Fine-tuning batch 10 (epoch 1)
Fine-tuning batch 11 (epoch 1)
Fine-tuning batch 12 (epoch 1)
Fine-tuning batch 13 (epoch 1)
Fine-tuning batch 14 (epoch 1)


In [6]:
def predict(prompt):
    gradient = Gradient()
    # Load the pre-fine-tuned model adapter using the saved ID or state
    model_adapter = gradient.get_model_adapter(model_adapter_id=model_id)

    sample_query = f"### Instruction: {prompt} \n\n### Response:"
    completion = model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    return completion

